In [51]:
#В качестве датасета выбран набор данных о винах: https://www.kaggle.com/datasets/zynicide/wine-reviews
#нужно создать модели МО, используя регрессию, классификацию, кластеризацию

# Загружаем библиотеки
import numpy as np # работа с векторами
import matplotlib.pyplot as plt # рисовать графики
import pandas as pd # для работы с табличная
import sklearn # машинное обучение на python
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [41]:
# загружаем данные
dataset = pd.read_csv('winemag-data_first150k.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   150930 non-null  int64  
 1   country      150925 non-null  object 
 2   description  150930 non-null  object 
 3   designation  105195 non-null  object 
 4   points       150930 non-null  int64  
 5   price        137235 non-null  float64
 6   province     150925 non-null  object 
 7   region_1     125870 non-null  object 
 8   region_2     60953 non-null   object 
 9   variety      150930 non-null  object 
 10  winery       150930 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 12.7+ MB


In [42]:
#Обработка данных
#в загруженных данных мы имеем следующие параметры/столбцы:
#[country, description, designation, points, price, province, region_1, region_2, variety, winery]
#Так как понятие описания вина очень специфичное и трудно поддается обработке, мы удалим данный столбец. 
#Также стоит удалить столбец region_2 из-за большого количество пропусков

dataset = dataset.drop(columns = ['description', 'region_2'])
#удаляем первый столбец. Нам не нужна нумерация
dataset = dataset.drop(dataset.columns[0], axis=1)

In [43]:
#такие поля как designation, province, region_1, variety, winery имеют категориальный вид и с ними будет сложно работать. Нужно их нормализовать

# Создаем экземпляр LabelEncoder
label_encoder = LabelEncoder()

dataset['country_encoded'] = label_encoder.fit_transform(dataset['country'])
dataset['name'] = label_encoder.fit_transform(dataset['designation'])
dataset['province_encoded'] = label_encoder.fit_transform(dataset['province'])
dataset['region'] = label_encoder.fit_transform(dataset['region_1'])
dataset['variety_encoded'] = label_encoder.fit_transform(dataset['variety'])
dataset['winery_encoded'] = label_encoder.fit_transform(dataset['winery'])


dataset = dataset.drop(columns = ['country','designation', 'province', 'region_1', 'variety', 'winery'])


In [60]:
#переименуем в человеческий вид
new_column_names = {
    'country_encoded': 'country',
    'province_encoded': 'province',
    'variety_encoded': 'variety',
    'winery_encoded': 'winery'
}
dataset.rename(columns=new_column_names, inplace=True)

In [80]:
#нормализация данных
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
dataset = pd.DataFrame(scaled_data, columns=dataset.columns)

Пустые значения
Они встречаются только в name, price, region. Проверить это можно через print(dataset['points'].isnull().any())
Значение price можно попробовать восстановить используя КК, на основе параметра points. А вот пустые строки у name и region лучше удалить, что бы снизить значение предсказания в будущем

In [78]:
# Вычислим коэффициент корреляции
#correlation = dataset['points'].corr(dataset['price'])
# Заполним пропуски в столбце 'price' на основе значений в столбце 'points' с учетом коэффициента корреляции
#dataset['price'].fillna(dataset['points'] * correlation, inplace=True)

Коэффициент корреляции показал страшные значения, поэтому лучше удалим. Ниже можно глянуть общий разброс этих значений
31    0.014373
32    0.229932
33    0.007840
34    0.016551
35    0.057056
36    0.024390
37    0.010889
38    0.008275
39    0.010889

In [81]:
#удалим все пустые строки
dataset = dataset.dropna()